## ppr.py


In [ ]:
#----------------------------------------------
# Create Features by use 
# Dispersion Entropy and Permutation entropy
#----------------------------------------------

import pandas   as pd
import numpy   as np
from utility import entropy_dispersion, entropy_prmuta

# Load  parameters Entropy
def conf_entropy():    

    return(opt,d,tau,c)

# Load Data
def load_data(nFile):

# Obtain entropy : dispersión and Permutation
def gets_entropy(x,opt,d,tau,c):
    
    return()
    
# Obtain Features by use Entropy    
def gets_features():
    
    return(F)    
# Beginning ...
def main():
    conf_entropy()            
    load_data()
    F1 = gets_features()
    F2 = gets_features()
    F  = np.concatenate((F1, F2), axis=0)
    save_data(F)
    
       
if __name__ == '__main__':   
	 main()



## trn.py


In [ ]:
# Logistic Regression's Training :

import numpy      as np
import utility    as ut

#Save weights and Cost
def save_w_cost():
    ...
    return
#
def iniWs():
    ...
    return(W,V)
# Load config train for Regression
#
#Training by use mGD
def train():    
    
    ....
    return()
# Load data to train 
def load_data():
    
    
    return()
#
def conf_train(nFile):
    ....
    return()

# Beginning ...
def main():    
    conf_train()
    load_data()   
    train()             
    save_w_cost(W,Cost, 'pesos.csv','costo.csv')
       
if __name__ == '__main__':   
	 main()



## tst.py

In [ ]:
# Testing for Logistic Regresion
import numpy as np

def forward(xv,w):
    ...
    return(zv)
#
def measure(yv,zv):
    ...
    return(cmatrix,Fscores)
#
def save_measure(cm,Fsc,nFile1,nFile2):
    ...
    return()
# Load weight
def load_w(nFile):
    ...
    return(W)
# 
def load_data(nFile):
    ....
    return(x,y)
# Beginning ...
def main():			
	load_data()
	load_w()
	zv     = forward(xv,W)      		
	cm,Fsc = metricas(yv,zv) 	
	save_measure(cm,Fsc,'cmatrix.csv','Fscores.csv')		

if __name__ == '__main__':   
	 main()



## utility.py

In [ ]:

# My Utility : auxiliars functions

import pandas as pd
import numpy  as np


# CResidual-Dispersion Entropy
def entropy_dispersion(x,d,tau,c):
    ...
    return(entr)
#
# Permutation Entropy
def entropy_permuta(x,m,tau):
    ...
    return(entr)
#


